In [1]:
import os
from utils import *
import numpy as np
import shutil

In [2]:

def find_files(target_folder, suffix):
    
    conf_files = []  # 记录 config
    
    for root, dirs, files in os.walk(target_folder):
        for file in files:
            if file.endswith(suffix):
                conf_files.append(os.path.join(root, file))

    return conf_files


In [3]:
def p_open(cmd):
    p = subprocess.Popen(cmd,shell=True,stdout=subprocess.PIPE)
    return p

In [4]:
def edit_distance_bit_bytes(b1, b2):
    m, n = len(b1), len(b2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j
        
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if b1[i-1] == b2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j],    
                                   dp[i][j-1],    
                                   dp[i-1][j-1])  
    return dp[m][n]


In [5]:
# local_target_info = ["tcp","127.0.0.1",4288]
target_info = ["tcp","172.20.10.6",12345] # black-box wingman

In [6]:
target_folder = '../dicom/conf'
conf_files = find_files(target_folder,'.conf')
print(f"Found .conf files: {conf_files}")

Found .conf files: ['../dicom/conf/dcmtk_v3.6.5.conf', '../dicom/conf/dcmtk_v3.6.5_Dec.conf', '../dicom/conf/dcmtk_v3.6.7.conf', '../dicom/conf/dcmtk_v3.6.6.conf']


In [7]:

#print(f"Found .raw files: {in_raw}")

In [8]:
wingman_list = []
for conf in conf_files:
    with open(conf,'r') as f:
        content = f.readlines()
        w = wingman()
        if content[0].startswith('NAME:'):
            w.name = content[0].strip().split(': ')[1]
        if content[1].startswith('BOOT:'):
            w.boot = content[1].strip().split(': ')[1]
        if content[2].startswith('CLOSE:'):
            w.close = content[2].strip().split(': ')[1]
        if content[3].startswith('AFLNET_COMMAND:'):
            w.afl = content[3].strip().split(': ')[1]
        if w.isInited():
            print(w.afl)
            wingman_list.append(w)

../aflnet/afl-fuzz -d -i ../dicom/in/ -o ../dicom/out/dcmtk_v3.6.5/ -N tcp://127.0.0.1/4291 -P DICOM -D 10000 -q 3 -s 3 -E -K -R ../dicom/repo/storescp_v3.6.5
../aflnet/afl-fuzz -d -i ../dicom/in/ -o ../dicom/out/dcmtk_v3.6.5_Dec/ -N tcp://127.0.0.1/4292 -P DICOM -D 10000 -q 3 -s 3 -E -K -R ../dicom/repo/storescp_v3.6.5_Dec
../aflnet/afl-fuzz -d -i ../dicom/in/ -o ../dicom/out/dcmtk_v3.6.7/ -N tcp://127.0.0.1/4289 -P DICOM -D 10000 -q 3 -s 3 -E -K -R ../dicom/repo/storescp_v3.6.7
../aflnet/afl-fuzz -d -i ../dicom/in/ -o ../dicom/out/dcmtk_v3.6.6/ -N tcp://127.0.0.1/4290 -P DICOM -D 10000 -q 3 -s 3 -E -K -R ../dicom/repo/storescp_v3.6.6


In [9]:

#sq = msg_sq()
#sq.msg = b'\x1b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [10]:
def create_dir(target):
    if os.path.exists(target):
        shutil.rmtree(target)
        print(f"Clean f{target}")
    os.makedirs(target)
    print(f"Create f{target}")

In [11]:
def afl_start(wing, cur_round, out_dir_list):
    cmd = wing.afl.strip().split(" ")
    out = os.path.join(cmd[cmd.index("-o")+1], f"round_{cur_round}")
    cmd[cmd.index("-o")+1] = out
    #print(" ".join(cmd))
    
    create_dir(out)
    out_dir_list.append(out)
    afl = subprocess.Popen(" ".join(cmd), shell=True,stdout = subprocess.PIPE)
    
    return afl

In [12]:
def afl_end(afl, sleep_time, wing):
    time.sleep(sleep_time)
    afl.terminate()
    afl.wait()
    c = subprocess.run(f"sudo pkill -9 -f dicom/out/{wing.name}",shell=True,stdout = subprocess.PIPE)

In [13]:
def find_all_files(directory):
    files_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            files_list.append(os.path.join(root,file))
    return files_list

In [14]:
def find_files_in_root_directory(directory):
    files_list = []

    for entry in os.listdir(directory):
        full_path = os.path.join(directory, entry)
        if os.path.isfile(full_path):
            files_list.append(entry)
    
    return files_list

In [15]:
def copy_file(source, destination):
    shutil.copy(source,destination)
    print(f"Seed Update --- {source}")

In [16]:
def send_signal(signal):
    host = "localhost"
    port = 25
    
    message = signal
    
    client.send(message.encode())
    client.close()
    
    print(f"Signal {signal} Send --")


In [17]:
round_time = 60 * 2 * 1    # 1 round = 2 hours

# Iterate 5 rounds
for cur_round in range(1,6):
    print(f"\033[0;36;m[INFO] ============ Round {cur_round} ============\033[0m")
    out_dir_list = []
    
    # Allocating Energy
    print("\033[0;36;m[INFO] +++++++ Allocating Energy +++++++\033[0m")
    in_dir = "../dicom/in"
    in_raw = find_files(in_dir,'.raw')
    seed_corpus = []        # seed corpus is the list of meg_sq
    for seed in in_raw:     # reload the input seeds
        with open(seed,'rb') as f:
            content = f.read1()
            sq = msg_sq()
            sq.msg = content
            seed_corpus.append(sq)
    
    
    '''
      Get wingman's cali_response_list from wingman,
      for the first round resp distance calculation
    '''
    for w in wingman_list:
        w.start()
        response = []
        for seed in seed_corpus:
            response = seed.send(target_info, w.name)
            w.cur_cali_res_list.append(response)
        w.shutdown()
    
    
    # get cali_response_list from target   
    #client.send("pause".encode())
    target_cali_res_list = []
    resposne = []
    for seed in seed_corpus:
        response = seed.send(target_info, "Target")
        target_cali_res_list.append(response)
    
    time.sleep(2)
    
    #client.send("continue".encode())
    target_cali_res = b""
    for res in target_cali_res_list:
        target_cali_res = target_cali_res + res
    
    distances = []
    for w in wingman_list:
        wingman_cur_cali_res = b""
        for res in w.cur_cali_res_list:
            wingman_cur_cali_res = wingman_cur_cali_res + res
        w.cur_distance = edit_distance_bit_bytes(target_cali_res, wingman_cur_cali_res)
        distances.append(w.cur_distance)
    distances = np.array(distances)
    
    weights = 1 / np.maximum(distances, 1)
    weights[distances == 0] = weights.max()

    allocation = (weights / weights.sum()) * round_time
    print("\033[0;32;m[INFO] Allocation:", allocation, "seconds\033[0m")

    # wingmates fuzzing
    print("\033[0;36;m[INFO] +++++++ Wingmate Fuzzing +++++++\033[0m")
    for i in range(0, len(wingman_list)):
        print(f"\033[0;32;m[INFO] fuzzing on {wingman_list[i].name} start ...\033[0m")
        wingman_list[i].start()
        cur_afl = afl_start(wingman_list[i], cur_round, out_dir_list)
        afl_end(cur_afl, round(allocation[i]), wingman_list[i])
        print(f"\033[0;32;m[INFO] fuzzing on {wingman_list[i].name} end ...\033[0m")   

    # update the seed_corpus
    print("\033[0;32;m[INFO] +++++++ Updating Seed Corpus +++++++\033[0m")
    temp = wingman_list[0].afl.strip().split(" ")
    in_dir = temp[temp.index("-i") + 1]
    in_files = find_files_in_root_directory(in_dir)
    update_list = []
    for out_dir in out_dir_list:
        out_files = find_files_in_root_directory(os.path.join(out_dir,"queue"))
        if len(in_files) != len(out_files):
            print(out_files)
            for out in out_files:
                if "orig:" not in out:
                    update_list.append(os.path.join(out_dir,"queue",out))
    
    print("\033[0;32;m[INFO] New seeds found:", len(update_list),"\033[0m")
    #client.send("pause".encode())

    # Connect to blackbox
    client = socket.socket()
    client.connect((target_info[1], 12345))

    i = 0
    for file in update_list:
        with open(file, 'rb') as f:
            b_msg = f.read1()
            tem = ''.join('\\x{:02x}'.format(byte) for byte in b_msg)
            byte_msg_tem = bytes(f"msg|{tem}", "utf-8")
            client.send(byte_msg_tem)
        
        prefix = file.strip().split("/")[3]
        suffix = file.strip().split("/")[-1]
        seed_file_path = os.path.join(in_dir,f"round_{cur_round}_new_seed_No.{i}.{prefix}.{suffix}.raw")
        if os.path.exists(seed_file_path):
            os.system(r"touch {}".format(seed_file_path))
            copy_file(file, seed_file_path)

        i = i + 1

    #client.send("reload".encode())
    client.send("stop".encode())
    client.close()


[INFO] ============ Round 1 ============
[INFO] +++++++ Allocating Energy +++++++
dcmtk_v3.6.5 start ---
dcmtk_v3.6.5 close msg : Command 'sudo pkill -9 -f dicom/repo/storescp_v3.6.5' died with <Signals.SIGKILL: 9>.
dcmtk_v3.6.5_Dec start ---
dcmtk_v3.6.5_Dec close msg : Command 'sudo pkill -9 -f dicom/repo/storescp_v3.6.5_Dec' died with <Signals.SIGKILL: 9>.
dcmtk_v3.6.7 start ---
dcmtk_v3.6.7 close msg : Command 'sudo pkill -9 -f dicom/repo/storescp_v3.6.7' died with <Signals.SIGKILL: 9>.
dcmtk_v3.6.6 start ---
dcmtk_v3.6.6 close msg : Command 'sudo pkill -9 -f dicom/repo/storescp_v3.6.6' died with <Signals.SIGKILL: 9>.
[INFO] Allocation: [30. 30. 30. 30.] seconds
[INFO] +++++++ Wingmate Fuzzing +++++++
[INFO] fuzzing on dcmtk_v3.6.5 start ...
dcmtk_v3.6.5 start ---
Clean f../dicom/out/dcmtk_v3.6.5/round_1
Create f../dicom/out/dcmtk_v3.6.5/round_1
[INFO] fuzzing on dcmtk_v3.6.5 end ...
[INFO] fuzzing on dcmtk_v3.6.5_Dec start ...
dcmtk_v3.6.5_Dec start ---
Clean f../dicom/out/dcmtk_v